In [98]:
import pandas as pd

dataset = pd.read_csv('transfusion.data')#carga el dataset,titulos
dataset.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [102]:
dataset.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [103]:
y = dataset['whether he/she donated blood in March 2007']
x = dataset.drop(columns=['whether he/she donated blood in March 2007'])

print('Targets = \n',y)
print('Inputs = \n',x)

Targets = 
 0      1
1      1
2      1
3      1
4      0
      ..
743    0
744    0
745    0
746    0
747    0
Name: whether he/she donated blood in March 2007, Length: 748, dtype: int64
Inputs = 
      Recency (months)  Frequency (times)  Monetary (c.c. blood)  Time (months)
0                   2                 50                  12500             98
1                   0                 13                   3250             28
2                   1                 16                   4000             35
3                   2                 20                   5000             45
4                   1                 24                   6000             77
..                ...                ...                    ...            ...
743                23                  2                    500             38
744                21                  2                    500             52
745                23                  3                    750             62
746         

In [104]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import numpy as np

kf = KFold(n_splits=4, shuffle = True)
scores = []
C_values = [1.0,1.5,2.5,5,10,100,1000]
Gamma_values = [0.1]
results=dict()

for gamma_value in Gamma_values:
    for c_value in C_values:

        for train_index, test_index in kf.split(x,y): 
            x_train, x_test = x.iloc[train_index,:], x.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            scaler =  StandardScaler()
            scaler.fit(x_train)
            x_trian = scaler.transform(x_train)
            x_test = scaler.transform(x_test)

            model = SVC(C=c_value)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)

            score = f1_score(y_test, y_pred, average='macro')
            #print('F1 macro: ',score)
            scores.append(score)

            #score = f1_score(y_test, y_pred, average='weighted')
            #print('F1 w: ',score)
            #print('Train index',train_index)
            #print('Test index',test_index)
            #print('\n')

        key = "C ={:.3f}, Gamma= {:.3f}".format(c_value,gamma_value)
        results[key] = [np.mean(np.array(scores)),np.std(np.array(scores))]

    
results

{'C =1.000, Gamma= 0.100': [0.43244590387993764, 0.003960560539664139],
 'C =1.500, Gamma= 0.100': [0.4324329522365524, 0.004792784706152942],
 'C =2.500, Gamma= 0.100': [0.43238259953813807, 0.007173439243096357],
 'C =5.000, Gamma= 0.100': [0.43235559279845126, 0.008210668336001696],
 'C =10.000, Gamma= 0.100': [0.43235503692115407, 0.008217324591447686],
 'C =100.000, Gamma= 0.100': [0.43233531646360834, 0.008870846291076581],
 'C =1000.000, Gamma= 0.100': [0.4323293561532751, 0.00906100107923644]}

In [105]:
pd.DataFrame(results)

,"C =1.000, Gamma= 0.100","C =1.500, Gamma= 0.100","C =2.500, Gamma= 0.100","C =5.000, Gamma= 0.100","C =10.000, Gamma= 0.100","C =100.000, Gamma= 0.100","C =1000.000, Gamma= 0.100"
0,0.432446,0.432433,0.432383,0.432356,0.432355,0.432335,0.432329
1,0.003961,0.004793,0.007173,0.008211,0.008217,0.008871,0.009061


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'criterion':['gini','entropy'],'splitter':['best','random'], 'max_depth':np.linspace(2,10,9),
              'min_samples_split':np.linspace(000.1,1,100)}

model = DecisionTreeClassifier()

gridSearch = GridSearchCV(model, param_grid, cv=5)
gridSearch.fit(x,y)

gridSearch.best_params_

# Modelo BagginClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier

param_grid = {'n_estimators':[10,20,30,40,50,60,70,80,90,100], 'criterion':['gini','entropy'], 'max_depth':[None,5,10,20]}

model = BagginClassifier()

gridSearch = GridSearchCV(model, param_grid, cv=4)
gridSearch.fit(x,y)

gridSearch.best_params_

# Arbol de decision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'criterion':['gini','entropy'], 'splitter':['best','random'], 'max_depth':np.linspace(2,10,9),
              'min_samples_split':np.linspace(000.1,1,100)}

model = DecisionTreeClassifier()

gridSearch = GridSearchCV(model, param_grid, cv=5)
gridSearch.fit(x,y)

gridSearch.best_params_

# Modelo AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

param_grid = {'n_estimators':[10,15,20,25,30,35,40,45,50], 'learning_rate':np.linspace(1,5,20), 'algorithm':['SAMME','SAMME.R']}

model = AdaBoostClassifier()

gridSearch = GridSearchCV(model, param_grid, cv=4)
gridSearch.fit(x,y)

gridSearch.best_params_